# Финальный проект. A/B-тестирование

**Цель исследования**: провести оценку результатов A/B-теста изменений, связанных с внедрением улучшенной рекомендательной системы.

**Задачи исследования**:
- оценить корректность проведения теста:
   - соответствие данных требованиям технического задания
   - время проведения теста
   - аудиторию теста
- проанализировать результаты теста

**Материалы исследования:** Датасет с действиями пользователей, техническое задание и вспомогательные датасеты. 

**Техническое задание**:
- Название теста: `recommender_system_test`;
- группы: А — контрольная, B — новая платёжная воронка;
- дата запуска: 2020-12-07;
- дата остановки набора новых пользователей: 2020-12-21;
- дата остановки: 2021-01-04;
- аудитория: 15% новых пользователей из региона EU;
- назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- ожидаемое количество участников теста: 6000.
- ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
    - конверсии в просмотр карточек товаров — событие `product_page`,
    - просмотры корзины — `product_cart`,
    - покупки — `purchase`.

**Описание данных**

1. `ab_project_marketing_events.csv` — календарь маркетинговых событий на 2020 год.  

Структура файла:
- `name` — название маркетингового события;
- `regions` — регионы, в которых будет проводиться рекламная кампания;
- `start_dt` — дата начала кампании;
- `finish_dt` — дата завершения кампании.


2. `final_ab_new_users.csv` — пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.  

Структура файла:
- `user_id` — идентификатор пользователя;
- `first_date` — дата регистрации;
- `region` — регион пользователя;
- `device` — устройство, с которого происходила регистрация.


3. `final_ab_events.csv` — действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.  

Структура файла:
- `user_id` — идентификатор пользователя;
- `event_dt` — дата и время события;
- `event_name` — тип события;
- `details` — дополнительные данные о событии. Например, для покупок, `purchase,` в этом поле хранится стоимость покупки в долларах.


4. `final_ab_participants.csv` — таблица участников тестов.  

Структура файла:
- `user_id` — идентификатор пользователя;
- `ab_test` — название теста;
- `group` — группа пользователя.

**Этапы исследования**:
1. Исследование данных:
2. Оценка корректности проведения теста
3. Исследовательский анализ данных
4. Оценка результатов A/B-тестирования
5. Выводы по этапу исследовательского анализа данных и по проведённой оценке результатов A/B-тестирования. 
Общее заключение о корректности проведения теста.

## Изучение данных и подготовка их к анализу
    • Проверка соответствия данных в столбцах заданному типу
    • Проверка на пропуски и дубликаты. Описание причин появления в данных пропусков и аномалий

In [33]:
#импорт библиотек
import pandas as pd
import scipy.stats as stats
import numpy as np
import math as mth
import datetime as dt
from matplotlib import pyplot as plt

import seaborn as sns
import plotly.figure_factory as ff
from plotly import graph_objects as go
import plotly.express as px

In [34]:
#загрузка данных
try:
    ab_events, marketing_events, new_users, participants = (
    pd.read_csv('/datasets/final_ab_events.csv'),  #  действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.
    pd.read_csv('/datasets/ab_project_marketing_events.csv'),  # календарь маркетинговых событий на 2020 год.
    pd.read_csv('/datasets/final_ab_new_users.csv'),  # пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.
    pd.read_csv('/datasets/final_ab_participants.csv')  # таблица участников тестов.
)
except:
    ab_events, marketing_events, new_users, participants = (
    pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_events.csv'),  #  действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.
    pd.read_csv('https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv'),  # календарь маркетинговых событий на 2020 год.
    pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_new_users.csv'),  # пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.
    pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_participants.csv') # таблица участников тестов.
)

In [35]:
#выгрузка основной информации о датасетах

pd.options.display.max_colwidth = 100

for i in [ab_events, marketing_events, new_users, participants]:
    print()
    print('Первые строки таблицы:')
    display(i.head());
    print()
    print('Основная информация о таблице:')
    display(i.info());
    print()
    print('Проверка таблицы на пропуски:')
    display(i.isna().sum());
    print()
    print('Проверка таблицы на дубликаты:')
    display(i.duplicated().sum());
    print('-'*50)
    print()


Первые строки таблицы:


,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99



Основная информация о таблице:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


None


Проверка таблицы на пропуски:


...


Проверка таблицы на пропуски:


user_id    0
group      0
ab_test    0
dtype: int64


Проверка таблицы на дубликаты:


0

--------------------------------------------------



In [36]:
# рассчитаем количество уазанных дополнительных данных относительно каждого события
ab_events.groupby('event_name')['details'].count().reset_index()

,event_name,details
0,login,0
1,product_cart,0
2,product_page,0
3,purchase,62740


**Вывод**:

В таблце `ab_events`:
- `event_dt` — необходимо привести к типу datetime
- `details` — пропуски в 377 577 строках. Заполнено 62 740 строк для события purchase. По остальным событиям доп. данные не указаны. Столбец не используется в дальнейшем анализе, поэтому пропуски оставим как есть.  
- дубликатов нет

В таблце `marketing_events`:
- `start_dt`  — необходимо перевести к типу datetime
- `finish_dt`  — необходимо перевести к типу datetime
- пропусков нет
- дубликатов нет

В таблце `new_users`:
- `first_date`  — необходимо перевести к типу datetime
- пропусков нет
- дубликатов нет

В таблце `participants`:
- данные соотвествуют типу
- пропусков нет
- дубликатов нет



In [37]:
# приведение столбцов с датами к типу datetime
ab_events['event_dt'] = pd.to_datetime(ab_events['event_dt'])
marketing_events['start_dt'] = marketing_events['start_dt'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
marketing_events['finish_dt'] = marketing_events['finish_dt'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
new_users['first_date'] = new_users['first_date'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))

## Оценка корректности проведения теста
- Соответствие данных всем требованиям технического задания.   
- Время проведения теста  
- Аудитория теста  

1. Оценка негативного влияния нарушений требований на результаты теста.
2. Контроль количества уникальных пользователей, принимающих участие в тесте  при проверке условий технического задания и фильтрации данных для детального изучения влияния каждого условия на аудиторию теста.

### Соответствие данных всем требованиям технического задания. 

    • Название теста: `recommender_system_test`;
    • группы: А — контрольная, B — новая платёжная воронка;
1. дата запуска: 2020-12-07;
    • дата остановки набора новых пользователей: 2020-12-21;
2. дата остановки: 2021-01-04;
3. аудитория: 15% новых пользователей из региона EU;
    • назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
4. ожидаемое количество участников теста: 6000.
5. ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:  
    ◦ конверсии в просмотр карточек товаров — событие `product_page`  
    ◦ просмотры корзины — `product_cart`  
    ◦ покупки — `purchase`  

#### Даты запуска и остановки набора новых пользователей
- дата запуска: 2020-12-07;
- дата остановки набора новых пользователей: 2020-12-21;
    
Объединим таблицы `new_users` и `participants` для сверки дат набора новых пользователей

In [38]:
test_users = participants.merge(new_users, on='user_id', how='left') 
test_users = test_users.query('ab_test == "recommender_system_test"')
display(test_users.head())
print('Дата начала регистрации:', test_users.first_date.min().strftime('%Y-%m-%d'))
print('Дата остановки набора новых пользователей:', test_users.first_date.max().strftime('%Y-%m-%d'))

,user_id,group,ab_test,first_date,region,device
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC
1,A7A3664BD6242119,A,recommender_system_test,2020-12-20,EU,iPhone
2,DABC14FDDFADD29E,A,recommender_system_test,2020-12-08,EU,Mac
3,04988C5DF189632E,A,recommender_system_test,2020-12-14,EU,iPhone
4,482F14783456D21B,B,recommender_system_test,2020-12-14,EU,PC


Дата начала регистрации: 2020-12-07
Дата остановки набора новых пользователей: 2020-12-21


**Даты запуска и остановки набора** новых пользователей соответствует ТЗ

#### Дата остановки теста - 2021-01-04
Посмотрим последнюю дату действий новых пользователей 

In [39]:
ab_events.event_dt.max().strftime('%Y-%m-%d')

'2020-12-30'

**Логи есть до 30 декабря** при том, что заявленная дата остановки теста - 4 января.   
Это значит, что по пользователям, зарегистрировавшим после 18 декабря, нет полных наблюдейний за 14 дней из-за чего результаты теста могут быть искажены

#### Aудитория: 15% новых пользователей из региона EU

In [40]:
new_users.head()

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


In [41]:
# запишем даты проведения теста в переменные
test_start_date = pd.to_datetime('2020-12-7')
registration_end_date =pd.to_datetime('2020-12-21')
test_finish_date = pd.to_datetime('2021-1-4')

all_new_users_region = new_users.query('@test_start_date < first_date < @test_finish_date').groupby('region')['user_id'].count().reset_index()
print('Распределение всех новых пользователей по регионам:')
display(all_new_users_region)

test_new_users_region = test_users.groupby('region')['user_id'].count().reset_index()
test_new_users_region['%']=round(test_new_users_region['user_id']/all_new_users_region['user_id']*100)
print('Распределение пользователей из теста recommender_system_test по регионам:')
display(test_new_users_region)
print('% новых пользователей из региона EU:',test_new_users_region.loc[2,'%'])

Распределение всех новых пользователей по регионам:


,region,user_id
0,APAC,2898
1,CIS,2862
2,EU,42065
3,N.America,8313


Распределение пользователей из теста recommender_system_test по регионам:


,region,user_id,%
0,APAC,72,2.0
1,CIS,55,2.0
2,EU,6351,15.0
3,N.America,223,3.0


% новых пользователей из региона EU: 15.0


**В аудиторию теста** должны входить 15% новых пользователей из региона EU. 
Таких пользователей в данных также оказалось 15%, что соответствует ТЗ.

#### Ожидаемое количество участников теста

In [42]:
# объединение таблицы с логами пользователей с таблицей с пользователями из тестовых групп
test_users_merged = test_users.merge(ab_events, on='user_id', how='left') 
display(test_users_merged.head())
print('Количество событий в объединенной таблице:', test_users_merged.shape[0])
print('Количество уникальных пользователей в объединенной таблице:', test_users_merged['user_id'].nunique())

,user_id,group,ab_test,first_date,region,device,event_dt,event_name,details
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,purchase,99.99
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-25 00:04:56,purchase,4.99
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:29,product_cart,NaN
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-25 00:04:57,product_cart,NaN
4,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,product_page,NaN


Количество событий в объединенной таблице: 27724
Количество уникальных пользователей в объединенной таблице: 6701


**Количество уникальных пользователей** - 6701, что соответсвует ТЗ

#### Ожидаемый эффект
За 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
- просмотр карточек товаров — событие `product_page`,
- просмотры корзины — событие `product_cart`,
- покупки — событие `purchase` 

Отберем события пользователей в рамках 14 дней с момента регистрации. Сравним в контрольной и тестовой группе 

In [43]:
#удаляем события, старше 14 дней после регистрации
test_users_merged['lifetime'] = test_users_merged['event_dt']-test_users_merged['first_date']
test_users_merged['lifetime'] = test_users_merged['lifetime'].astype('timedelta64[D]')
test_users_merged = test_users_merged.query('lifetime <= 14')
print('Количество событий после фильтрации по лайфтайму:', test_users_merged.shape[0])
print('Количество уникальных пользователей после фильтрации по лайфтайму:', test_users_merged['user_id'].nunique())

Количество событий после фильтрации по лайфтайму: 24070
Количество уникальных пользователей после фильтрации по лайфтайму: 3675


In [44]:
# распределение событий по тестовым группам
groups_funnel = test_users_merged.pivot_table(index='event_name', columns='group', values='user_id', aggfunc='nunique').reset_index()
groups_funnel1 = groups_funnel.reindex([0,2,1,3])
groups_funnel1

group,event_name,A,B
0,login,2747,927
2,product_page,1780,523
1,product_cart,824,255
3,purchase,872,256


In [45]:
# визуализация последовательной воронки в разрезе групп
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'group_A',
    orientation = "h",
    y = groups_funnel1['event_name'],
    x = groups_funnel1["A"],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'group_B',
    orientation = "h",
    y = groups_funnel1['event_name'],
    x = groups_funnel1["B"],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.update_layout(
        title='Доля пользователей переходящих на следующий шаг воронки в разрезе групп')
fig.show()

По воронке видим:
- просмотр карточек товаров (product_page) у группы А составляет 65%, у группы В - 56%. Разница -9%
- просмотры корзины (product_cart) у группы А составляет 30%, у группы В - 28%. Разница -2%
- покупки (purchase) у группы А составляет 32%, у группы В - 28%. Разница -4%

Таким образом, за 14 дней с момента регистрации пользователи из тестовой группы не показали улучшение метрик на 10%. Ожидаемый эффект за счет улучшения рекомендательной системы не достигнут.

Отметим, что количество пользователей совершивших покупку больше, чем просмотревших корзину. Это значит, что на сайте реализована функция покупки товаров прямо из карты товаров и пользователю не обязательно заходить в корзину.

### Время проведения теста
Период проведения нашего теста: с 07 декабря 2020г. по 04 января 2021г. Дата остановки набора новых пользователей: 2020-12-21   
Посмотрим, имеются ли пересечения периода проведения теста с маркетинговыми активностями. Для этого построим диаграмму Ганта

In [46]:
marketing_events = marketing_events.rename(columns={'name': 'Task', 'start_dt': 'Start', 'finish_dt': 'Finish'}) 
fig = ff.create_gantt(marketing_events)
fig.show()

In [47]:
print('Пересечение периода проведения теста с маркетинговыми активностями')
marketing_events1 = marketing_events.query('not(Start >= @test_finish_date or Finish <=@test_start_date)')
marketing_events1

Пересечение периода проведения теста с маркетинговыми активностями


,Task,regions,Start,Finish
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
10,CIS New Year Gift Lottery,CIS,2020-12-30,2021-01-07


**Вывод:** С 07 декабря по 21 декабря, в период набора новых пользователей в тестовые группы, маркетинговых активностей не было.

На диаграмме Ганта видно, что в декабре-январе проводилось 2 промо:
- Christmas&New Year Promo с 25 декабря по 03 января в EU, N.America
- CIS New Year Gift Lottery	с 30 декабря по 07 января в CIS

Промо наложились на вторую половину периода проведения теста. Согласно ТЗ, учитываются пользователи из EU, то есть на тестовую и контрольную группы могло повлиять Christmas&New Year Promo с 25 декабря по 03 января. Это необходимо будет учесть при оценке результатов A/B-тестирования.

### Аудитория теста 
- не пересекается с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. 
- Проверка равномерности распределения по тестовым группам и правильность их формирования.

In [48]:
#проверка на пользователей, попавших в обе группы теста recommender_system_test
group_nunique = participants.query('ab_test== "recommender_system_test"').groupby('user_id').agg({'group':'nunique'}).reset_index()
print('Пользователи, которые попали в обе группы теста recommender_system_test:',len(group_nunique.query('group > 1')))

Пользователи, которые попали в обе группы теста recommender_system_test: 0


In [49]:
# проверка на пользователей, участвующих в нескольких тестах
print('Пользователи участвующие в обоих тестах:', len(participants.groupby('user_id')['ab_test'].nunique().reset_index().query('ab_test > 1')))

Пользователи участвующие в обоих тестах: 1602


In [50]:
print('Доля пользователей, участвующих в 2х тестах:')
both_tests = participants.groupby('ab_test')['user_id'].count().reset_index()
both_tests['%']=round(len(participants.groupby('user_id')['ab_test'].nunique().reset_index().query('ab_test > 1'))/both_tests['user_id']*100)
both_tests

Доля пользователей, участвующих в 2х тестах:


,ab_test,user_id,%
0,interface_eu_test,11567,14.0
1,recommender_system_test,6701,24.0


Всего в тесте `recommender_system_test` участвуют 6 701 пользователь. Из них, 1602 пользователя (24%) также попали в тест другой тест. Посмотрим как распределены данные пользователи в обеих тестах по группам А (контрольной) и Б (тестовой) 

In [51]:
#пользователи, попавшие в оба теста
both_test_participants = participants.groupby('user_id')['ab_test'].nunique().reset_index().query('ab_test > 1')['user_id']

#пользователи из 2х тестов, попавшие в тестовую группу B теста interface_eu_test:
interface = participants.query(
    'ab_test== "interface_eu_test" and group == "B"').loc[participants['user_id'].isin(both_test_participants)]['user_id']

In [52]:
print('Распределние всех пользователей теста interface_eu_test по группам:')
interface_eu_test = participants.query('ab_test== "interface_eu_test"').groupby('group')['ab_test'].count().reset_index()
interface_eu_test['%']=round(interface_eu_test['ab_test']/both_tests.loc[0, 'user_id']*100)
display(interface_eu_test)

print('Распределение пользователей, участвующих в 2х тестах, в interface_eu_test:')
both_test_interface = participants.query(
    'ab_test== "interface_eu_test"').loc[participants['user_id'].isin(both_test_participants)].groupby(
    'group')['user_id'].count().reset_index()
# добавление столбца с расчётом доли
both_test_interface['%'] = round(both_test_interface['user_id']/participants.query('ab_test== "interface_eu_test"')['user_id'].count()*100)
both_test_interface

Распределние всех пользователей теста interface_eu_test по группам:


,group,ab_test,%
0,A,5831,50.0
1,B,5736,50.0


Распределение пользователей, участвующих в 2х тестах, в interface_eu_test:


,group,user_id,%
0,A,819,7.0
1,B,783,7.0


In [53]:
print('Распределние всех пользователей теста recommender_system_test по группам:')
recommender_system_test = participants.query('ab_test== "recommender_system_test"').groupby('group')['ab_test'].count().reset_index()
recommender_system_test['%']=round(recommender_system_test['ab_test']/both_tests.loc[1, 'user_id']*100)
display(recommender_system_test)

print('Распределение пользователей, участвующих в 2х тестах, в recommender_system_test:')
#распределение пользователей в тесте 'recommender_system_test'
both_test_recommender = participants.query(
    'ab_test== "recommender_system_test"').loc[participants['user_id'].isin(both_test_participants)].groupby(
    'group')['user_id'].count().reset_index()
# добавление столбца с расчётом доли
both_test_recommender['%'] = round(both_test_recommender['user_id']/participants.query('ab_test== "recommender_system_test"')['user_id'].count(),2)
display(both_test_recommender)

print('Распределение пользователей тестовой группы interface_eu_test, в тесте recommender_system_test:')
recommender = participants.query('ab_test== "recommender_system_test"').loc[participants['user_id'].isin(interface)].groupby(
    'group')['user_id'].count().reset_index()
recommender['%']=round(recommender['user_id']/both_test_interface.loc[1, 'user_id']*100)
recommender

Распределние всех пользователей теста recommender_system_test по группам:


,group,ab_test,%
0,A,3824,57.0
1,B,2877,43.0


Распределение пользователей, участвующих в 2х тестах, в recommender_system_test:


,group,user_id,%
0,A,921,0.14
1,B,681,0.10


Распределение пользователей тестовой группы interface_eu_test, в тесте recommender_system_test:


,group,user_id,%
0,A,439,56.0
1,B,344,44.0


**Вывод:** По таблице распределения пользователей тестовой группы interface_eu_test, в тесте recommender_system_test видим, что доли (А = 56%, В = 44%) практически совпадают с общим распределением, где А = 57% и В = 43%. 

Таким образом, пользователи тестовой группы конкурирующего теста равномерно распределены в тесте recommender_system_test и одинаково влияют на контрольную и тестовую группы нашего теста recommender_system_test. Поэтому удалять таких пользователей не будем.

### Вывод

При оценке корректности проведения теста мы увидели, что аудитория пересекается с конкурирующим тестом (1602 пользователя). Так как пользователи группы B теста изменений интерфейса равномерно распределены в нашем тесте улучшений рекомендательной системы, то они одинаково влияют на нашу контрольную и тестовые группы, поэтому решено оставить таких пользователей.

Время проведения теста пересекается с маркетинговой активностью Christmas&New Year Promo, что могло повлиять на поведение пользователей с 25 декабря.

Дата запуска и остановки набора новых пользователей соответствует  техническому заданию. Но при этом, до даты остановки теста, 04 января 2021г., событий не хватает — последний лог датирован 30 декабря. Возможно произошел сбой в выгрузке данных. Это также могло повлиять на результаты теста

В аудиторию теста входият 15% новых пользователей из региона EU, как и заявлено в ТЗ. 

Количество уникальных пользователей, попавших в тест составляет 6701, что соответствует ТЗ об ожидании 6000 участников теста

Количество пользователей по группам распределены неравномерно, то есть изменения смотрятся на малой тестовой выборке (в контрольно группе А 2747 пользователей, в тестовой группе В — 927 пользователей). За 14 дней с момента регистрации пользователи из группы В не показали улучшение метрик, в сравнении в контрольной группой показатели выглядят хуже. 

По воронке видно, что не все пользователи переходят в корзину для покупки, а могут это сделать прямо со страницы просмотра карточек товаров  

## Исследовательский анализ данных:
### Равномерность распределения количества событий на пользователя в выборках
Проверим одинаково ли распределены в выборках количество событий на пользователя 

In [54]:
#количество событий пользователя групп теста
events_per_user = test_users_merged.pivot_table(index=['user_id','group'], values = 'event_name', aggfunc='count').reset_index()
events_per_user.head()

,user_id,group,event_name
0,001064FEAAB631A1,B,6
1,0010A1C096941592,A,12
2,00341D8401F0F665,A,2
3,003DF44D7589BBD4,A,15
4,00505E15A9D81546,A,5


In [55]:
fig = px.histogram(events_per_user, x="event_name", color="group", color_discrete_sequence= ['#B279A2','#72B7B2'])
fig.update_layout(
    title_text='Распределение пользователей по количеству событий ', 
    xaxis_title_text='Количество событий', 
    yaxis_title_text='Количество пользователей')

fig.show()

In [56]:
round(events_per_user.groupby('group')['event_name'].describe().reset_index())

,group,count,mean,std,min,25%,50%,75%,max
0,A,2747.0,7.0,4.0,1.0,4.0,6.0,9.0,24.0
1,B,928.0,6.0,3.0,1.0,3.0,4.0,8.0,24.0


Количество событий на пользователя в выборках распределено неравномерно. По графику видим, что у пользователей группы А больше событий. В среднем на 1 пользователя группы А приходится 7 событий, группы В - 5 событий. 

### Распределение числа событий в выборках по дням 

In [57]:
#количество событий в тестовых группах по дням 
test_users_merged['event_dt'] = test_users_merged['event_dt'].dt.date
events_per_date = test_users_merged.pivot_table(index=['event_dt','group'], values = 'event_name', aggfunc='count').reset_index()
events_per_date.head()

,event_dt,group,event_name
0,2020-12-07,A,331
1,2020-12-07,B,378
2,2020-12-08,A,341
3,2020-12-08,B,252
4,2020-12-09,A,385


In [58]:
fig = px.bar(events_per_date, x="event_dt", y ='event_name', color="group", barmode='group', color_discrete_sequence= ['#B279A2','#72B7B2'])
fig.update_layout(
    title_text='Распределение числа событий в выборках по дням', 
    xaxis_title_text='Даты', 
    yaxis_title_text='Количество событий')

fig.show()

По графику видим, что в первые 7 дней теста распределение числа событий в выборках было сопоставимым, но 14 декабря в группе А резко возросло количество событий и они далее растут до 21 декабря до пикового количества 1995 действий. После этого количество событий стало сокращаться. По группе В резких перепадов не наблюдается, но после 21 декабря график событий тоже идет на спад. 

Больше количество событий можно связать с попаданием проведения теста в предновогодние недели, на которых активность пользователей всегда растет. Но в группе В всплеска не наблюдается, поэтому к определяющему фактору отнести предпразничные закупки не можем. 

Судя по тому, что снижение активности по обеим группам совпадает с днём остановки набора новых пользователей, можно предположить, что всплекс событий в группе А связан с регистрацией новых пользователей и их попаданием в основном в группу А, а не в группу В. Посмотрим как распределены данные по регистрации пользователей

In [59]:
#количество событий в тестовых группах по дням 
first_date = test_users_merged.pivot_table(index=['first_date','group'], values = 'user_id', aggfunc='nunique').reset_index()
first_date.head()

,first_date,group,user_id
0,2020-12-07,A,163
1,2020-12-07,B,182
2,2020-12-08,A,98
3,2020-12-08,B,44
4,2020-12-09,A,82


In [60]:
fig = px.bar(first_date, x="first_date", y ='user_id', color="group", barmode='group', color_discrete_sequence= ['#B279A2','#72B7B2'])
fig.update_layout(
    title_text='Регистрация пользователей в выборках по дням', 
    xaxis_title_text='Даты', 
    yaxis_title_text='Количество событий')

fig.show()

На графике набора пользователей видно, что пик регистрации пользователей приходится на 14 и 21 декабря, причем большая часть пользователей попадает в контрольную группу. Это коррелируется со всплеском числа событий в группе А в те же даты.

### Изменение конверсии в воронке в выборках на разных этапах

In [61]:
# визуализация последовательной воронки в разрезе групп
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'group_A',
    orientation = "h",
    y = groups_funnel1['event_name'],
    x = groups_funnel1["A"],
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name = 'group_B',
    orientation = "h",
    y = groups_funnel1['event_name'],
    x = groups_funnel1["B"],
    textposition = "inside",
    textinfo = "value+percent previous"))

fig.update_layout(
        title='Доля пользователей переходящих на следующий шаг воронки в разрезе групп')
fig.show()

Рассмотрим воронку событий более подробно на предмет изменения конверсии на разных этапах.

Воронка состоит из четырех этапов:
1. заход пользователя на сайт - событие login
2. просмотр карточек товаров — событие product_page,
3. просмотры корзины — событие product_cart,
4. покупка — событие purchase

Больше всего пользователей теряются на 2х этапах:
- на этапе просмотра карточек: в группе А -35%, в группе В - 44% от зашедших на сайт
- на этапе просмотра корзины: в группe А только 46% переходят из второго этапа в третий, в группе В показатель чуть лучше - 49% пользователей просмотревших карту товара переходят в корзину.   

Ранее мы отметили, что пользователи могут совершить покупку из карты товаров, минуя третий этап, поэтому просмотревших корзину даже меньше чем совершивших покупку. 

В целом, от первого события до оплаты в контрольной группе дошло 32% пользователей, а в экспериментальной группе только 28%. Улучшение метрик не наблюдается

### Особенности данных, которые нужно учесть, прежде чем приступать к A/B-тестированию

При оценке корректности проведения теста мы увидели, что дата запуска и остановки набора новых пользователей соответствует  техническому заданию. Но при этом, до даты остановки теста, 04 января 2021г., событий не хватает — последний лог датирован 30 декабря. Возможно, произошел сбой в выгрузке данных. Это могло повлиять на результаты теста

Количество уникальных пользователей, попавших в тест составляет 6701, что соответствует ТЗ об ожидании 6000 участников теста. Однако, после отбора событий в течение 14 дней после регистрации, пользователей остается только  3675человек.

Время проведения теста пересекается с маркетинговой активностью Christmas&New Year Promo, что могло повлиять на поведение пользователей с 25 декабря.

Аудитория пересекается с конкурирующим тестом (1602 пользователя). Так как пользователи группы B теста изменений интерфейса равномерно распределены в нашем тесте улучшений рекомендательной системы, то они одинаково влияют на нашу контрольную и тестовые группы, поэтому решено оставить таких пользователей.

Количество пользователей по группам распределены неравномерно, то есть изменения смотрятся на малой тестовой выборке (в контрольно группе А - 2747 пользователей, в тестовой группе В — 927 пользователей). 

В период набора новых пользователей, большинство попадает в контрольную группу А. Пики регистрации приходятся на начало недели 7,14,21 декабря, что может быть связано с закупкой рекламы. Из-за действий новых пользователей и наблюдаются всплески количества событий 14 и 21 декабря. При этом в тестовой группе В всплески не наблюдаются, события распределены более равномерно. В среднем на 1 пользователя группы А приходится 7 событий, группы В - 5 событий. 

За 14 дней с момента регистрации пользователи из группы В не показали улучшение метрик на целевое значение 10%. В сравнении в контрольной группой показатели выглядят хуже. При этом по воронке замечено, что не все пользователи переходят в корзину для покупки, а могут это сделать прямо со страницы просмотра карточек товаров.  

## Оценка результатов A/B-тестирования
Рассчитаем долю пользователей из контрольной и тестовой групп, совершивших каждое действие.  
Так как на первом этапе `login` в обеих группах конверсия 100%, исключим шаг из проверки.  
Таким образом, будет проведен множественный тест из 3х проверок

In [62]:
# количество пользователей на каждом шаге
groups_funnel = test_users_merged.pivot_table(index='event_name', columns='group', values='user_id', aggfunc='nunique').reset_index()
groups_funnel1 = groups_funnel.query('event_name != "login"').reindex([2,1,3])
groups_funnel1

group,event_name,A,B
2,product_page,1780,523
1,product_cart,824,255
3,purchase,872,256


In [63]:
# количество пользователей в группе
users_in_group = test_users_merged.groupby('group').agg({'user_id':'nunique'}).T
users_in_group

group,A,B
user_id,2747,928


### Проверка статистической разницы долей z-критерием.

Проверим, является ли  отличие между группами статистически достоверным.

Сформулируем гипотезы:

**Нулевая гипотеза**: в сраниваемых группах, между долями пользователей, совершивших событие, разницы нет.  
**Альтернативная гипотеза**: в сраниваемых группах между долями пользователей, совершивших событие, есть значимая разница.

* alpha - за уровень статистической значимости примем 1%

Так как предполагается множественная проверка гипотез, применим метод Шидака для корректировки уровня значимости, чтобы снизить вероятность ложнопозитивного результата.

In [64]:
alpha = 0.05
shidak_alpha = round(1 - (1 - alpha)**(1/3),5)
print('Уровень значимости с поправкой Шидака:', shidak_alpha)

Уровень значимости с поправкой Шидака: 0.01695


In [65]:
from statsmodels.stats.proportion import proportions_ztest 

def proportions(c1, c2, n1, n2):
    count=[c1, c2] 
    nobs=[n1, n2]
    
    stat, p_value = proportions_ztest(count=count,nobs=nobs, alternative='two-sided')
    
    print('p-значение события {} : {}'.format(groups_funnel['event_name'][i], p_value))
    
    if (p_value < shidak_alpha):
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными') 

In [66]:
for i in groups_funnel1.index:
    proportions(groups_funnel1.loc[i,'A'], groups_funnel1.loc[i,'B'],users_in_group['A'][0],users_in_group['B'][0])
    print()

p-значение события product_page : 4.310980554712425e-06
Отвергаем нулевую гипотезу: между долями есть значимая разница

p-значение события product_cart : 0.14534814557238196
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

p-значение события purchase : 0.017592402663314678
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными



Результаты A/B-тестирования говорят о том, что:
- в сраниваемых группах, между долями пользователей, совершивших события `product_cart` и `purchase`, отличия нельзя считать статистически достоверными (между долями разницы нет).
- а между долями пользователей групп А и В, совершивших событие `product_page` различие статистически значимо

### Выводы по оценке результатов A/B-тестирования.

При проведении множественного теста, мы применили поправку Шидака, чтобы снизить вероятность ложнопозитивного результата.
Тесты не выявили значимой разницы между группами на этапах: `product_cart` и `purchase`.
При этом, между долями пользователей групп А и В, совершивших событие `product_page` есть значимая разница.

В целом, выводы остаются прежними: пользователи тестовой группы В не показали улучшение конверсии на целевые 10%. У группы А конверсия выше. 

## Общее заключение о корректности проведения теста.

В данном проекте мы провели оценку результатов A/B-теста, направленного на тестирование изменений, связанных с внедрением улучшенной рекомендательной системы. 

На этапе исследования данных изменили тип данных в столбцах с датами.   
Также изучили найденные пропусков в дополнительных данных о событиях. На дальнейший анализ пропуски не влияют, строки решили не трогать.

При сверке данных с требованиями технического задания были выявлены следующие несоответствия, которые могли повлиять на результаты:
- не хватает данных с 30 декабря по 04 января, хотя заявленный период проведения теста должен охватывать даты с 07 декабря 2020г. по 04 января 2021г. 
- С 25 декабря на поведение пользователей мог повлиять запуск маркетинговой активности Christmas&New Year Promo
- аудитория пересекается с конкурирующим тестом, но пользователи из парралельной тетсовой группы равномерно распределены между группами в нашем тесте
- Количество пользователей по группам распределены неравномерно. В группе А почти в 3 раза больше пользователей, чем в группе В. 
- За 14 дней с момента регистрации пользователи из группы В не показали улучшение метрик. В сравнении в контрольной группой показатели выглядят хуже.
- Для совершения покупки пользователям не требуется переходить в корзину, есть возможность со страницы просмотра карточек товаров перейти сразу к оплате

Кроме этого, дальнейший анализ показал, что:
- Количество событий на пользователя в выборках распределено неравномерно.  В среднем на 1 пользователя группы А приходится 7 событий, группы В - 5 событий. 
- В разрезе недели активнее всего регистрация новых пользователей происходит в понедельники. 
- В группу А попадает больше новых пользователей, чем в группу В. 14 и 21 декабря регистрируется больше всего пользователей, что отражается резким ростом количества событий в группе А с 14 по 21 декабря  

Воронка состоит из четырех этапов:
1. заход пользователя на сайт - событие login
2. просмотр карточек товаров — событие product_page,
3. просмотры корзины — событие product_cart,
4. покупка — событие purchase

Больше всего пользователей отваливаются на втором и третьем шаге (просмотр карточек и просмотр корзины).
Конверсия зарегистрировавших пользователей в покупателей в группе А 32%, а в группе В — 28%. 

Проверка статистической разницы долей z-критерием выявила статистически значимое различие между долями пользователей групп А и В только для события просмотра карточек товаров (product_page)

Таким образом, улучшенная рекомендательная система не показала ожидаемый уровень улучшения метрик на 10%. Однако на результаты могли значительно повлиять указанные выше отклонения условий проведения теста. 

В данной ситуации стоит рассмотреть вариант проведения повторного теста в более изолированных условиях с соблюдением ТЗ. Однако заказчику исследования следует учесть фактор дороговизны и ресурсоёмкости проведения A/B-теста и оценить важность тестируемой фичи, поскольку в тестовой группе наблюдалось не недостаточное улучшение, а именно ухудшение показателей. 